In [12]:
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain.vectorstores import Chroma
embedding = OllamaEmbeddings(model="llama3")

In [13]:
persist_directory = './chroma/'

In [14]:
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [15]:
print(vectordb._collection.count())

53


In [16]:
from langchain_community.llms import Ollama
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [17]:
metadata_field_info = [
    AttributeInfo(
        name="Header",
        description="This is the header of the section from which this text originates",
        type="string",
    ),
    AttributeInfo(
        name="source",
        description="Path of the PDF file from which this chunk is derived. It is possible to observe the hash that symbolizes the name of the PDF, in addition to the number of questions it contains. The PDFs are medical scientific articles in the field of medicine. The path format is 'articles/hash(n).pdf', where 'hash' represents the PDF name, 'n' represents the number of related questions, and '.pdf' represents the file extension.",
        type="string",
    ),
]

document_content_description = "Articles"

In [18]:
llm = Ollama(
    model="llama3",
    temperature = 0
)

retriever = vectordb.as_retriever()

In [19]:
question = "What is the main subject of the article?"
docs_ss = vectordb.similarity_search(question,k=3)
len(docs_ss)

3

In [20]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from transformers import pipeline

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

section_counts = {
        "Introduction": 0,
        "Background": 0,
        "Methods": 0,
        "Results": 0,
        "Discussion": 0,
        "Conclusion": 0
    }

In [25]:
final_results = []
partial_result = {
            "article": "",
            "section_counts": section_counts
        }
new_article = True

results_outputs =[]

In [26]:
for q in questions:   
    article = "2dff5573c7185251e19b4a301173e8f8bc85347c.pdf"
    quizz_question = ['quizz_question']
    possible_answers = ['possible_answers']


    question = f"""The following question is based on the provided article. 
        Please read the article carefully and determine which section of the article the question pertains to. Justify your choice by referring to specific parts of the article.

        Article: {article}

        Question: {quizz_question}
        Possible answers: {possible_answers}

        Identify the relevant section of the article and provide a brief justification."""

    template = """You will be provided with an article and a question related to it. 
    Your task is to read the article and determine the section that best answers the question. Provide a concise and specific justification for your choice.

    {context}

    Question: {question}

    Helpful Answer:"""
    QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

    qa = RetrievalQA.from_chain_type(
        llm,
        retriever=retriever,
        memory=memory,
        return_source_documents=False,
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}

    )

    result = qa({"query": question}, return_only_outputs=True)
    results_outputs.append(result['result'])

    # Carregar o pipeline de classificação de texto
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

    # Definir o texto e as categorias
    text = result['result']

    # Definir as seções como categorias
    labels = ["Introduction", "Methods", "Results", "Discussion", "Conclusion"]

    # Classificar o texto
    classification = classifier(text, labels)

    # Obter a seção mais provável
    predicted_section = classification['labels'][0]
    section_counts[predicted_section] += 1

final_results.append({
        "article": "2c5bee41e4ae9aa65206b113c7b84c8152cd4aed.pdf",
        "section_counts": section_counts
    })
print(final_results)

C:\Users\fiorentc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


[{'article': '2c5bee41e4ae9aa65206b113c7b84c8152cd4aed.pdf', 'section_counts': {'Introduction': 0, 'Background': 0, 'Methods': 0, 'Results': 0, 'Discussion': 15, 'Conclusion': 0}}]


In [27]:
print(results_outputs)

["I'm happy to help!\n\nAfter carefully reading the provided article, I believe that the question pertains to Section 1 of the article. Here's why:\n\nThe article starts with a reference list (Section 1) that provides background information on various studies and research related to colorectal cancer screening and prevention. The references are cited throughout the article, including in the introduction, methods, results, and discussion sections.\n\nGiven that the question is based on the provided article, I think it's likely that the question is related to one of these referenced studies or concepts introduced in Section 1.", "I'm happy to help!\n\nAfter carefully reading the provided article, I believe that the question pertains to Section 1 of the article. Here's why:\n\nThe article starts with a reference list (Section 1) that provides background information on various studies and research related to colorectal cancer screening and prevention. The references are cited throughout th